In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import os
family_group = "6.Heliconiinae_Danainae_Nymphalinae"
annotated_genome_location = f"/mnt/h/My Drive/Circadian Rhythm Genes Project/7.Timeless Exon Analysis/{family_group}/1.Blast_result"
species_list = os.listdir(annotated_genome_location)
if "desktop.ini" in species_list:
    species_list.remove("desktop.ini")
# species_list = ["Boloria_euphrosyne","Boloria_selene","Brenthis_daphne","Brenthis_hecate","Brenthis_ino","Dryadula_phaetusa","Dryas_iulia_moderata","Eueides_isabella","Fabriciana_adippe","Heliconius_charithonia","Heliconius_nattereri","Heliconius_sara","Philaethria_dido","Hestina_assimilis"]
# species_list = ["Limenitis_camilla"]
# species_list = ["Argynnis_bischoffii_washingtonia","Boloria_euphrosyne","Boloria_selene","Brenthis_daphne","Brenthis_hecate","Brenthis_ino","Dryadula_phaetusa","Dryas_iulia_moderata","Eueides_isabella","Fabriciana_adippe","Heliconius_charithonia","Heliconius_nattereri","Heliconius_sara","Philaethria_dido","Hestina_assimilis"]
# species_list = ["Bicyclus_anynana","Coenonympha_glycerion","Elymnias_hypermnestra","Erebia_aethiops","Erebia_ligea","Hipparchia_semele","Lasiommata_megera","Maniola_hyperantus","Maniola_jurtina","Melanargia_galathea","Oeneis_ivallda","Pararge_aegeria"]
# species_list = ["Erebia_ligea","Hipparchia_semele","Lasiommata_megera","Maniola_hyperantus","Maniola_jurtina","Melanargia_galathea","Oeneis_ivallda","Pararge_aegeria"]
# species_list = ["Vanessa_tameamea"]
for species in species_list:
    print(species)
    blast_location = f"/mnt/h/My Drive/Circadian Rhythm Genes Project/7.Timeless Exon Analysis/{family_group}/1.Blast_result"
    
    list_of_files = os.listdir(f"{blast_location}/{species}")
    query_species_list = []
    coordinate_detail_dictionary = {}
    coordinate_info_dictionary = {}
    scaffold = ''
    for file in list_of_files:
        if file.endswith(".csv") and len(file.split("_")) >= 5 and not(file.endswith("_old.csv")):
#             print(file)
            if "." in file[:-4]:
                query_species = f"{file.split('_')[-2].split('.')[1]}_{file.split('_')[-1].split('.')[0]}"
            else:
                query_species = f"{file.split('_')[-2]}_{file.split('_')[-1].split('.')[0]}"
            query_species_list.append(query_species)
            coordinate_detail_dictionary[query_species] = {}
            with open(f"{blast_location}/{species}/{file}", 'r') as coordinate_file_open:
                coordinate_file_lines_list  =  coordinate_file_open.readlines()
            if scaffold == '':
                scaffold = coordinate_file_lines_list[1].split(",")[1]
    
            for i in range(1,len(coordinate_file_lines_list)):
                
                coordinate_file_lines_split = coordinate_file_lines_list[i].split(",")
                start,stop = coordinate_file_lines_split[2],coordinate_file_lines_split[3]
                exon = f"Exon_{i}"
                if exon not in coordinate_info_dictionary:
                    coordinate_info_dictionary[exon] = {}
                coordinate_info_dictionary[exon][query_species] = [start,stop]
                coordinate_detail_dictionary[query_species][exon] = coordinate_file_lines_list[i]
            
            # break
            # print(query_species)
            # print(file)
    # print(coordinate_info_dictionary)
    # assert False
    output = coordinate_file_lines_list[0]
    for exon,species_specific_coordinates in coordinate_info_dictionary.items():
        # print(exon,species_specific_coordinates)
        coordinates_list = []
        previous_coordinates = ""
        match = 1
        for query_species, coordinates in coordinate_info_dictionary[exon].items():
            coordinates_list.append([query_species,coordinates])
            if previous_coordinates != '':
                if coordinates != previous_coordinates:
                    match = 0
                    
            previous_coordinates = coordinates
       
        
        
        if match == 0:
            print(exon,"mismatch")
            print(coordinates_list)
                
            while True:
                choice = input("Proceed?")
                if choice[0].lower() == "y" or choice[0].lower() == "n":
                    break
            
            while True:
                if choice[0].lower() == "y":
                    alignment_detail_dictionary = {}
                    for query_species_name in query_species_list:
                        list_of_folders  = os.listdir(f"{blast_location}/{species}")
                        for folders in list_of_folders:
                            if folders.endswith(query_species_name):
                                # print()
                                
                                with open(f"{blast_location}/{species}/{folders}/{exon}/for_blast/new_query.txt", 'r') as query_file_open:
                                    # alignment_detail = query_file_open.readlines()[0]
                                    alignment_detail = query_file_open.readlines()[0].split("set")[1][:-1]
                                    alignment_detail_dictionary[query_species_name] = alignment_detail
                                with open (f"{blast_location}/{species}/{folders}/{exon}/for_alignment/{exon}_translated_genomic_sequence_{alignment_detail}.fa.hat2", 'r') as alignment_score_file:
                                    distance_score = float(alignment_score_file.readlines()[-1].rstrip())
                                try:
                                    with open (f"{blast_location}/{species}/{folders}/{exon}/for_blast/new_query_spliced.txt", 'r') as new_query_file:
                                        new_query = (new_query_file.readlines()[-1].rstrip())
                                except:
                                    new_query = "Issue with query"
                                print(query_species_name,exon, alignment_detail, distance_score,f"\n{new_query}")
                    while True:            
                        check_alignment = input("Check Alignment??")
                        if check_alignment[0].lower() == "y" or check_alignment[0].lower() == "n":
                            break
                    if check_alignment[0].lower() == "y":
                        for query_species_name in query_species_list:
                            list_of_folders  = os.listdir(f"{blast_location}/{species}")
                            for folders in list_of_folders:
                                if folders.endswith(query_species_name):
                                    with open (f"{blast_location}/{species}/{folders}/{exon}/for_alignment/alignment_clustal_{exon}_translated_genomic_sequence_{alignment_detail_dictionary[query_species_name]}.fa.txt", 'r') as clustal_alignment_file:
                                        clustal_alignment = clustal_alignment_file.readlines()
                                    print("".join(clustal_alignment))
                                    
                                # assert False
                    while True:
                        try:
                            species_number = (input(f"Choose Species 1 - {len(query_species_list)}"))
                            
                            
                            if species_number[0].lower() == 'n':
                                output +=  f'{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n'
                                break 
                            output += coordinate_detail_dictionary[query_species_list[int(species_number)-1]][exon]
                            break
                        except:
                            print("Retry")
                    break
                elif choice[0].lower() == "n":
                    output +=  f'{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n'
                    break
                
                
        else:
            print(exon, "Match")
            print(coordinates_list)
            output += coordinate_detail_dictionary[query_species_list[0]][exon]
        
       

        # assert False
    # print(coordinate_detail_dictionary)
    print(output)
    with open(f"{blast_location}/{species}/final_coordinates.csv", "w") as out_file:
        out_file.write(output)
    
    # 'Coenonympha_glycerion,OY979626.1,7015875,7015963,0,N,5.Bicyclus_anynana_XM_024088150.2_Frame_0_rightoh_2_query_Exon_1,1,29,29\n'

Hestina_assimilis
Exon_1 Match
[['Danaus_plexippus', ['8131918', '8132014']], ['Vanessa_tameamea', ['8131918', '8132014']], ['Nymphalis_io', ['8131918', '8132014']]]
Exon_2 Match
[['Danaus_plexippus', ['8132083', '8132636']], ['Vanessa_tameamea', ['8132083', '8132636']], ['Nymphalis_io', ['8132083', '8132636']]]
Exon_3 Match
[['Danaus_plexippus', ['8132971', '8133124']], ['Vanessa_tameamea', ['8132971', '8133124']], ['Nymphalis_io', ['8132971', '8133124']]]
Exon_4 mismatch
[['Danaus_plexippus', ['8133212', '8133522']], ['Vanessa_tameamea', ['8133212', '8133468']], ['Nymphalis_io', ['8133212', '8133441']]]
Proceed?y
Danaus_plexippus Exon_4 31_frame0 1.242 
SNHSSPMVTSDPTSDSSDTGACGKINENPNPDINGWENSTGNVDDGNQSVQTNTENYKMPGDMSAGHENNLHHQSEVAPPDIKKMVSIHFLNNISDGYKFKDK
Vanessa_tameamea Exon_4 31_frame0 1.0 
SNHSSPMVTSDPTSDSSDTGACGKINENPNPDINGWENSTGNVDDGNQSVQTNTENYKMPGDMSAGHENNLHHQSEVAPPDIKKM
Nymphalis_io Exon_4 31_frame0 1.059 
SNHSSPMVTSDPTSDSSDTGACGKINENPNPDINGWENSTGNVDDGNQSVQTNTENYKMPGDMSAGHENN

Nymphalis_io Exon_15 16_frame0 1.607 
TIIPDDQDNNTLEDKAMCEDNDVCGVSDILCGDNDELPMDESCLLEPPVSPLVLPTIITMKHETGHFNLGRAESEYFNTN
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Danaus_plexippu SIL---------------------------------------------------------
set16_frame0    LICNVVIIPNRNVCLTDFKNVILFNTKFLFYNCYFYIKVELVFRTIIPDDQDNNTLEDKA
                 *                                                          

Danaus_plexippu -----------------TEEVVVSKSSVEAPPVAVLPPALVLPSAINMKQDGSGYVN---
set16_frame0    MCEDNDVCGVSDILCGDNDELPMDESCLLEPPVS----PLVLPTIITMKHE-TGHFNLGR
                                 .:*: :.:*.:  ***:    .****: *.**:: :*:.*   

Danaus_plexippu NDSEYN------------------------------------------------------
set16_frame0    AESEYFNTNRYVLFKNLAVFYVITVGKQAHVPPDACALPIRGIGSKERNIGIERLVRTTH
                 :***                                                       

Danaus_plexippu --------------
set16_frame0    DEEGSIFFLFGTLL
                              

CLUSTAL format alig

Proceed?y
Danaus_plexippus Exon_15 45_frame1 0.0 
SILTEEVVVSKSSVEAPPVAVLPPALVLPSAINMKQDGSGYVNNDSEYN
Vanessa_tameamea Exon_15 45_frame1 1.649 
SILTEEVVVS
Nymphalis_io Exon_15 45_frame1 1.712 
SILTEEVVVSKSSVEAPPVAVLPPALVL
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Danaus_plexippu -----SILTEEVVVSKSSVEAPPVAVLPPALVLPSAINMKQDGSGYVNNDSEYN
set45_frame1    YFSYRSILTEEVVVSKSSVEAPPVAVLPPALVLPSAINMKQDGSGYVNNDSEYN
                     *************************************************

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Vanessa_tameame SIIPVEDDTMKDLSPQE--ITHVPLEVTQLGTLESSSLILPTIMSVKQETSAFSTRLTDS
set45_frame1    YF------SYRSILTEEVVVSKSSVEAPPVAVL-PPALVLPSAINMKQDGSGYVN--NDS
                 :      : :.: .:*  ::: .:*.. :..* ..:*:**: :.:**: *.: .  .**

Vanessa_tameame DYYNN
set45_frame1    EY--N
                :*  *

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Nymphalis_io_XM -----SIISVENDTMKVCVPEKVTHVPLDMTSHQLGPLE--SSSLILPTIMSVNQEASAF
set45_frame1

Exon_1 Match
[['Danaus_plexippus', ['4996551', '4996647']], ['Vanessa_tameamea', ['4996551', '4996647']], ['Nymphalis_io', ['4996551', '4996647']]]
Exon_2 Match
[['Danaus_plexippus', ['4996747', '4997288']], ['Vanessa_tameamea', ['4996747', '4997288']], ['Nymphalis_io', ['4996747', '4997288']]]
Exon_3 Match
[['Danaus_plexippus', ['4997411', '4997564']], ['Vanessa_tameamea', ['4997411', '4997564']], ['Nymphalis_io', ['4997411', '4997564']]]
Exon_4 Match
[['Danaus_plexippus', ['4998166', '4998434']], ['Vanessa_tameamea', ['4998166', '4998434']], ['Nymphalis_io', ['4998166', '4998434']]]
Exon_5 mismatch
[['Danaus_plexippus', ['4998754', '4999034']], ['Vanessa_tameamea', ['4998751', '4999034']], ['Nymphalis_io', ['4998754', '4999034']]]
Proceed?y
Danaus_plexippus Exon_5 39_frame2 0.397 
DNNNITEDRENSNNVTNSEKKTGISETSDCGYGTQIENRESISTSSNEDDLPAKKPVHQKPHNAKQRVNNKVRSGITLQERKRKKIVKRSKAN
Vanessa_tameamea Exon_5 39_frame2 0.767 
QDNNNITEDRENSNNVTNSEKKTGISETSDCGYGTQIENRESISTSSNEDDLPAKKPVHQKPHNAKQRVNN

Proceed?y
Danaus_plexippus Exon_5 46_frame2 0.394 
DNNNITDDRENSKNITNSEKKTGISETSDCGYGTQIENRESISTSSNEDDLPAKKPVHQKPHNAKQRVNNKVRSGITLQERKRKKIVKRSKAN
Vanessa_tameamea Exon_5 46_frame2 0.765 
QDNNNITDDRENSKNITNSEKKTGISETSDCGYGTQIENRESISTSSNEDDLPAKKPVHQKPHNAKQRVNNKVRSGITLQERKRKKIVKRSKAN
Nymphalis_io Exon_5 46_frame2 0.909 
DNNNITDDRENSKNITNSEKKTGISETSDCGYGTQIENRESISTSSNEDDLPAKKPVHQKPHNAKQRVNNKVRSGITLQERKRKKIVKRSKAN
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Danaus_plexippu -----------DNNHISNDHSNSHNMKNNEKKTIISEPSDCGYGTQIENRESISTSSNED
set46_frame2    KIFYCNVFLKQDNNNITDDRENSKNITNSEKKTGISETSDCGYGTQIENRESISTSSNED
                           ***:*::*:.**:*:.*.**** ***.**********************

Danaus_plexippu ELPTTKPVHQKPHNTKQRTNNKVRAGVTLQDRKRKKIVKRGKAN----------------
set46_frame2    DLPAKKPVHQKPHNAKQRVNNKVRSGITLQERKRKKIVKRSKANMYVNNKTIPKYFLRFL
                :**:.*********:***.*****:*:***:*********.***                

Danaus_plexippu -----
set46_frame2    RVYLK
        

...

Choose Species 1 - 33
Exon_5 Match
[['Danaus_plexippus', ['7896456', '7896709']], ['Vanessa_tameamea', ['7896456', '7896709']], ['Nymphalis_io', ['7896456', '7896709']]]
Exon_6 Match
[['Danaus_plexippus', ['7895614', '7895974']], ['Vanessa_tameamea', ['7895614', '7895974']], ['Nymphalis_io', ['7895614', '7895974']]]
Exon_7 Match
[['Danaus_plexippus', ['7894202', '7894393']], ['Vanessa_tameamea', ['7894202', '7894393']], ['Nymphalis_io', ['7894202', '7894393']]]
Exon_8 Match
[['Danaus_plexippus', ['7893397', '7893497']], ['Vanessa_tameamea', ['7893397', '7893497']], ['Nymphalis_io', ['7893397', '7893497']]]
Exon_9 Match
[['Danaus_plexippus', ['7893036', '7893234']], ['Vanessa_tameamea', ['7893036', '7893234']], ['Nymphalis_io', ['7893036', '7893234']]]
Exon_10 Match
[['Danaus_plexippus', ['7892238', '7892440']], ['Vanessa_tameamea', ['7892238', '7892440']], ['Nymphalis_io', ['7892238', '7892440']]]
Exon_11 Match
[['Danaus_plexippus', ['7891374', '7891861']], ['Vanessa_tameamea', ['78913

Exon_1 Match
[['Danaus_plexippus', ['7295788', '7295884']], ['Vanessa_tameamea', ['7295788', '7295884']], ['Nymphalis_io', ['7295788', '7295884']]]
Exon_2 Match
[['Danaus_plexippus', ['7295979', '7296529']], ['Vanessa_tameamea', ['7295979', '7296529']], ['Nymphalis_io', ['7295979', '7296529']]]
Exon_3 Match
[['Danaus_plexippus', ['7296743', '7296896']], ['Vanessa_tameamea', ['7296743', '7296896']], ['Nymphalis_io', ['7296743', '7296896']]]
Exon_4 Match
[['Danaus_plexippus', ['7298223', '7298476']], ['Vanessa_tameamea', ['7298223', '7298476']], ['Nymphalis_io', ['7298223', '7298476']]]
Exon_5 Match
[['Danaus_plexippus', ['7298670', '7298956']], ['Vanessa_tameamea', ['7298670', '7298956']], ['Nymphalis_io', ['7298670', '7298956']]]
Exon_6 Match
[['Danaus_plexippus', ['7299410', '7299770']], ['Vanessa_tameamea', ['7299410', '7299770']], ['Nymphalis_io', ['7299410', '7299770']]]
Exon_7 Match
[['Danaus_plexippus', ['7300038', '7300229']], ['Vanessa_tameamea', ['7300038', '7300229']], ['Nymp

Proceed?y
Danaus_plexippus Exon_10 19_frame1 1.119 
DWSDLIEYVINTFIKKPYKMQNTVSLIEKEIFEDDKHLKEILLSAEELNPPLEEKK
Vanessa_tameamea Exon_10 19_frame1 0.0 
DWSDLIEYVINTFIKKPYKMQNTVSLIEKEIFEDDKHLKEILLSAEELNPPLEEKKSFQFSTEGDSN
Nymphalis_io Exon_10 19_frame1 0.437 
DWSDLIEYVINTFIKKPYKMQNTVSLIEKEIFEDDKHLKEILLSAEELNPPLEEKKSFQFSTEGDSN
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Danaus_plexippu DWSDLIEYVINTFIKKPYKLQNYSNIQLEMEIFDDDMKPKENLIDPE----PAPEIISVM
set19_frame1    DWSDLIEYVINTFIKKPYKMQNTVSL-IEKEIFEDDKHLKEILLSAEELNPPLEEKKSFQ
                *******************:**  .: :* ***:** : ** *:..*    *  *  *. 

Danaus_plexippu KKVNENENITD
set19_frame1    FSTEGDSN--R
                 ..: :.*   

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Vanessa_tameame DWSDLIEYVINTFIKKPYKMQNTVSLIEKEIFEDDKHLKEILLSAEELNPPLEEKKSFQF
set19_frame1    DWSDLIEYVINTFIKKPYKMQNTVSLIEKEIFEDDKHLKEILLSAEELNPPLEEKKSFQF
                ************************************************************

Van

Choose Species 1 - 32
Exon_11 Match
[['Danaus_plexippus', ['7375120', '7375610']], ['Vanessa_tameamea', ['7375120', '7375610']], ['Nymphalis_io', ['7375120', '7375610']]]
Exon_12 Match
[['Danaus_plexippus', ['7375892', '7376092']], ['Vanessa_tameamea', ['7375892', '7376092']], ['Nymphalis_io', ['7375892', '7376092']]]
Exon_13 Match
[['Danaus_plexippus', ['7376551', '7376731']], ['Vanessa_tameamea', ['7376551', '7376731']], ['Nymphalis_io', ['7376551', '7376731']]]
Exon_14 Match
[['Danaus_plexippus', ['7376820', '7376930']], ['Vanessa_tameamea', ['7376820', '7376930']], ['Nymphalis_io', ['7376820', '7376930']]]
Exon_15 Match
[['Danaus_plexippus', ['7377675', '7377866']], ['Vanessa_tameamea', ['7377675', '7377866']], ['Nymphalis_io', ['7377675', '7377866']]]
Exon_16 Match
[['Danaus_plexippus', ['7378530', '7378701']], ['Vanessa_tameamea', ['7378530', '7378701']], ['Nymphalis_io', ['7378530', '7378701']]]
Species,Scaffold,Start,Stop,Complement,Error,Gene,Query_start,Query_stop,Query_Lengt

Exon_16 Match
[['Danaus_plexippus', ['3762423', '3762594']], ['Vanessa_tameamea', ['3762423', '3762594']], ['Nymphalis_io', ['3762423', '3762594']]]
Species,Scaffold,Start,Stop,Complement,Error,Gene,Query_start,Query_stop,Query_Length
Vanessa_tameamea,NW_020663126.1,3754131,3754227,0,N,Danaus_plexippus_XM_032654785.2_Frame_0_rightoh_1_query_Exon_1,1,32,32
Vanessa_tameamea,NW_020663126.1,3754311,3754861,0,N,Danaus_plexippus_XM_032654785.2_Frame_2_rightoh_0_query_Exon_2,1,183,183,Y,N
Vanessa_tameamea,NW_020663126.1,3755050,3755203,0,N,Danaus_plexippus_XM_032654785.2_Frame_0_rightoh_1_query_Exon_3,1,51,51,Y,N
Vanessa_tameamea,NW_020663126.1,3755579,3755832,0,N,Danaus_plexippus_XM_032654785.2_Frame_2_rightoh_0_query_Exon_4,1,84,84,Y,N
Vanessa_tameamea,NW_020663126.1,3756340,3756623,0,N,Danaus_plexippus_XM_032654785.2_Frame_0_rightoh_2_query_Exon_5,1,94,94,Y,N
Vanessa_tameamea,NW_020663126.1,3757083,3757443,0,N,Danaus_plexippus_XM_032654785.2_Frame_1_rightoh_0_query_Exon_6,1,120,120,Y,N
Van

In [ ]:
1